In [46]:
import pandas as pd

# Import the seats data
df = pd.read_csv('data/cleaned/seats.csv')

vc = df['airline_iata'].value_counts()


# Filter to airline
airline = "UR"

# Filter to airline
df = df[df['airline_iata'] == airline]
# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'])
# Group by date and sum
df = df.groupby('date').sum()
df = df.reset_index()

# Add boolean for covid
df["covid"] = (df["date"] >= "2020-03-01") & (df["date"] <= "2021-6-30")
vc


/tmp/ipykernel_24569/2032176673.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('date').sum()


KeyError: 'airline_iata'

In [45]:
# Plug into Prophet
from prophet import Prophet

# Prepare the data for Prophet
df = df.rename(columns={'date': 'ds', 'seats': 'y'})


# Run the model
m = Prophet()
m.add_regressor('covid')
m.fit(df)

# Make a future dataframe
future = m.make_future_dataframe(periods=12, freq='MS')
future['covid'] = (future['ds'] >= "2020-03-01") & (future['ds'] <= "2021-6-30")


# Make predictions
forecast = m.predict(future)

# Plot the results
m.plot(forecast)


ValueError: Dataframe has less than 2 non-NaN rows.